# <center><h1>Companhia Aberta Demonstrativo Financeiro</h1></center>

# <h2>Load Libraries</h2>

In [91]:
import pandas as pd
import numpy as np

#<h2>Import Data</h2>

<h3>Load .csv</h3>

In [92]:
cia_aberta_df = pd.read_csv(r'dfp_cia_aberta_BPA_con_2020.csv', encoding='ISO-8859-1', sep=";")    # leio o .csv com os dados de todas as companhias abertas

<h3>Select Company</h3>

In [93]:
cia_aberta_df = cia_aberta_df[cia_aberta_df['CNPJ_CIA'] == '97.837.181/0001-47']    # dataframe = linhas do dataframe onde ('CNPJ_CIA' == '97.837.181/0001-47') == True
                                                                                    # (seleciono somente as linhas relativas a uma companhia de interesse)

In [94]:
df = cia_aberta_df[['ORDEM_EXERC','CD_CONTA','DS_CONTA','VL_CONTA']].copy()
df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']                                         # seleciono somente o último ou penúltimo exercício

In [95]:
df

,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA
56646,ÚLTIMO,1,Ativo Total,11498520.0
56648,ÚLTIMO,1.01,Ativo Circulante,4220022.0
56650,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,1.01.02,Aplicações Financeiras,0.0
56654,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...
56788,ÚLTIMO,1.02.04.02.07,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,1.02.04.02.08,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,1.02.04.02.09,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,1.02.04.02.10,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


#<h2>Wrangling</h2>

# <h3>Split CD_CONTA</h3>

In [96]:
df['CD_CONTA'].sort_values(ascending=True)

,CD_CONTA
56646,1
56648,1.01
56650,1.01.01
56652,1.01.02
56654,1.01.02.01
...,...
56788,1.02.04.02.07
56790,1.02.04.02.08
56792,1.02.04.02.09
56794,1.02.04.02.10


In [97]:
cd_conta_split = [string.split('.') for string in df['CD_CONTA']]

In [98]:
cd_conta_len = [len(lst) for lst in cd_conta_split]

In [99]:
num_levels = max(cd_conta_len)

In [100]:
num_levels

5

JOIN BASATA SULLA LISTA DI LISTE CON I VALORI DEI CODICI

In [101]:
cd_conta_split

[['1'],
 ['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01

In [102]:
first_column = [element[0] for element in cd_conta_split]       # row[0] for row in cd_conta_split?

In [103]:
key = first_column[0]

In [104]:
CD_DS_CONTA_dict[key]

'Ativo Total'

In [105]:
def join(key):
    ds_conta = CD_DS_CONTA_dict[key]
    return ds_conta

In [110]:
ds_conta_1 = [join(x) for x in first_column]

In [109]:
df.insert(2, column='CLASS_1', value=ds_conta_1)        # DS_CONTA_1 ao invés do que CLASS_1?

ValueError: cannot insert CLASS_1, already exists

In [111]:
df

,ORDEM_EXERC,CD_CONTA,CLASS_1,DS_CONTA,VL_CONTA
56646,ÚLTIMO,1,Ativo Total,Ativo Total,11498520.0
56648,ÚLTIMO,1.01,Ativo Total,Ativo Circulante,4220022.0
56650,ÚLTIMO,1.01.01,Ativo Total,Caixa e Equivalentes de Caixa,1728413.0
56652,ÚLTIMO,1.01.02,Ativo Total,Aplicações Financeiras,0.0
56654,ÚLTIMO,1.01.02.01,Ativo Total,Aplicações Financeiras Avaliadas a Valor Justo...,0.0
...,...,...,...,...,...
56788,ÚLTIMO,1.02.04.02.07,Ativo Total,Goodwill na aquisição da Caetex Florestal,8767.0
56790,ÚLTIMO,1.02.04.02.08,Ativo Total,Goodwill na aquisição da Cerâmica Urussanga em...,92944.0
56792,ÚLTIMO,1.02.04.02.09,Ativo Total,Goodwill na aquisição da Massima Revestimentos...,6110.0
56794,ÚLTIMO,1.02.04.02.10,Ativo Total,Goodwill na aquisição da Cecrisa Revestimentos...,168430.0


Drop Rows

2 Strategie:
- 1. Se esiste almeno un CD_CONTA che inizia per '1', oltre al primo, eliminare il primo
- 2. Filtro con due condizioni: CLASS == DS_CONTA per quella riga e il valore della riga sucessiva per CLASS è uguale a quello precedente.
Esempio: 'Ativo Total' (CLASS_1) == 'Ativo Total' (DS_CONTA) e allo stesso tempo 'Ativo Total' == 'Ativo Total' (prima e seconda riga di CLASS_1)

In [112]:
cd_conta_split

[['1'],
 ['1', '01'],
 ['1', '01', '01'],
 ['1', '01', '02'],
 ['1', '01', '02', '01'],
 ['1', '01', '02', '01', '01'],
 ['1', '01', '02', '01', '02'],
 ['1', '01', '02', '02'],
 ['1', '01', '02', '03'],
 ['1', '01', '03'],
 ['1', '01', '03', '01'],
 ['1', '01', '03', '01', '01'],
 ['1', '01', '03', '01', '02'],
 ['1', '01', '03', '02'],
 ['1', '01', '03', '02', '01'],
 ['1', '01', '04'],
 ['1', '01', '05'],
 ['1', '01', '06'],
 ['1', '01', '06', '01'],
 ['1', '01', '07'],
 ['1', '01', '08'],
 ['1', '01', '08', '01'],
 ['1', '01', '08', '02'],
 ['1', '01', '08', '03'],
 ['1', '02'],
 ['1', '02', '01'],
 ['1', '02', '01', '01'],
 ['1', '02', '01', '01', '01'],
 ['1', '02', '01', '02'],
 ['1', '02', '01', '03'],
 ['1', '02', '01', '04'],
 ['1', '02', '01', '04', '01'],
 ['1', '02', '01', '04', '02'],
 ['1', '02', '01', '05'],
 ['1', '02', '01', '06'],
 ['1', '02', '01', '07'],
 ['1', '02', '01', '07', '01'],
 ['1', '02', '01', '08'],
 ['1', '02', '01', '09'],
 ['1', '02', '01', '09', '01

Seleziona tutte le righe con un solo elemento: